In [1]:
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.recommendation.ALS

Intitializing Scala interpreter ...

Spark Web UI available at http://localhost:4040
SparkContext available as 'sc' (version = 2.4.6, master = local[*], app id = local-1632567354402)
SparkSession available as 'spark'


import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.recommendation.ALS


In [2]:
val ratings = spark.read.
    option("inferSchema","true").
    option("header","true").
    csv("Data/Movie_len/ratings.csv")

ratings: org.apache.spark.sql.DataFrame = [userId: int, movieId: int ... 2 more fields]


In [3]:
ratings.head()

res0: org.apache.spark.sql.Row = [1,1,4.0,964982703]


In [5]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [6]:
val Array(training, test) = ratings.randomSplit(Array(0.8, 0.2))
//트레이닝 데이터와 test 데이터 나누어 사용한다.
//머신 러닝 모델을 만들때 기초적인 사용 방향이다.

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userId: int, movieId: int ... 2 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userId: int, movieId: int ... 2 more fields]


In [7]:
training.count()

res3: Long = 80698


In [8]:
test.count()

res4: Long = 20138


In [9]:
training.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [10]:
test.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [13]:
val als = new ALS().
            setMaxIter(10).
            setRegParam(0.01).
            setUserCol("userId").
            setItemCol("movieId").
            setRatingCol("rating")

//ALS 오브젝트를 만들고 다양한 파라미터를 사용한다.
//이런 부분을 즉 파라미터 세팅 부분에 있어서는 스파크 ML ALS 문서를 확인하자.
//간단하게 보면 유저칼럼은 유저아이디에 , 아이템 칼럼은 무비아이디에 ,
//레이팅 칼럼은 레이팅에 잇다고 세팅을 해주는 것이다.

als: org.apache.spark.ml.recommendation.ALS = als_ebb5c52a80d7


In [14]:
val model = als.fit(training)
//우리가 모델을 학습 시킬떄는 만든 모델에 fit 이라는 것을 사용하고
//인자로는 데이터 셋을 넣어준다. 위에서 트레이닝 데이터 셋을 만들어 두었기에
//그걸 가지고 돌리는 것이다.

model: org.apache.spark.ml.recommendation.ALSModel = als_ebb5c52a80d7


In [15]:
val predictions = model.transform(test)
//학습이 끝났으면 학습이 끝난 모델에 test 셋을 넣는데 이떄 메소드 트랜스폼을
//사용한다. 이런 3가지 스텝이 스파크를 통한 데이터 분석시 꼭 거치는 스텝이다.


predictions: org.apache.spark.sql.DataFrame = [userId: int, movieId: int ... 3 more fields]


In [16]:
predictions.sort("userID").show(1000)
//그래서 결과를 보면 이처럼 나오고 있다.
//이게 테스트 데이터라 정답을 알고있고 얼마나 차이나는지 확인 가능하다.
//잘 예측한 경우도 있고 아닌 경우도 있다.
//당연한게 우리는 지금 스몰데이터 셋을 활용한 것이고 이터레이션을 많이 돌리지
//않았기에 그렇다.
//참고로 이터레이션은 반복횟수를 이야기 하는 것 같고 위의 setMaxIter 부분이
//최대 몇번까지 반복 즉 이터레이션을 할 것인지를 의미하는 파라미터인듯 하다.

+------+-------+------+----------+-----------+
|userId|movieId|rating| timestamp| prediction|
+------+-------+------+----------+-----------+
|     1|   1030|   3.0| 964982903|  3.0099783|
|     1|   1031|   5.0| 964982653|   4.055944|
|     1|   2366|   4.0| 964982462|  4.1702523|
|     1|   2000|   4.0| 964982211|  4.7795854|
|     1|    661|   5.0| 964982838|  2.8723927|
|     1|   2329|   5.0| 964983263|  4.8584166|
|     1|   2268|   4.0| 964982989|   4.945067|
|     1|   2395|   5.0| 964981093|  2.8964677|
|     1|   2959|   5.0| 964983282|  5.0101824|
|     1|    500|   3.0| 964981208|  4.5096984|
|     1|   3273|   5.0| 964983536|  4.7339435|
|     1|   1213|   5.0| 964982951|  4.9300046|
|     1|   1198|   5.0| 964981827|   5.378395|
|     1|   2054|   4.0| 964981747|  4.1684904|
|     1|    673|   3.0| 964981775|  2.2732587|
|     1|   2528|   3.0| 964982328|   4.258097|
|     1|   2078|   5.0| 964982838|  5.2642426|
|     1|   2033|   5.0| 964982903|   3.326647|
|     1|   22

|    16|    293|   4.0|1377477870|   3.945526|
|    16|    593|   3.0|1377476977|  3.8728046|
|    16|   1680|   3.5|1377476680|  3.1769385|
|    16|   7153|   3.5|1377477293|  3.9705646|
|    16|    319|   4.0|1377476675|  2.9251425|
|    16|   1247|   4.0|1377477535|  3.5922637|
|    16|   2109|   3.5|1377476690|  3.5337853|
|    16|    204|   2.0|1377476617|   2.624369|
|    17|  48394|   4.5|1322629142|  3.7408397|
|    17|   3681|   4.5|1307262300|   4.583225|
|    17|   2019|   3.5|1307262285|   4.051679|
|    17|   2717|   3.5|1305696306|   4.120883|
|    17|   1270|   4.5|1305696846|   4.317546|
|    17|  27773|   4.0|1322628790|    4.09983|
|    17|   1198|   4.5|1305696619|  4.4777293|
|    17|   5952|   5.0|1305696823|   4.636837|
|    17|  64839|   3.5|1322629112|  3.8085442|
|    17|   1732|   5.0|1322629209|  4.7087274|
|    17|   1234|   4.0|1305696795|   4.602774|
|    17|   1259|   4.0|1305696786|   4.183603|
|    17|    111|   4.0|1326943208|   4.186311|
|    17|    3

|    21|   2006|   2.5|1412968210|  3.5819168|
|    21|  81564|   2.5|1419794453|  3.1478605|
|    21|   8529|   4.5|1407617755|  4.0251083|
|    21|   2858|   2.0|1441392960|  2.5230606|
|    21|  79592|   3.5|1500701875|   2.632004|
|    21|   2378|   2.5|1376822727|  3.7845535|
|    21|   2671|   4.0|1417378450|  3.8486848|
|    21|   1265|   3.5|1428775428|  3.1211374|
|    21|  61160|   1.0|1468113607|  1.8500284|
|    21|   8368|   4.0|1412968057|   3.602532|
|    21|  33615|   4.0|1413055282|  3.1154091|
|    21| 122892|   3.5|1441392878|  2.6084747|
|    21|   1210|   4.5|1417378078|  3.5539255|
|    21|  44022|   3.5|1427558491|  2.3017995|
|    21|   5219|   3.0|1468113409|  2.4877594|
|    21|   7143|   4.0|1412967530|  2.0648887|
|    21| 109374|   3.5|1403460482|   2.594198|
|    21|   2541|   4.0|1376823011|   4.431809|
|    21|   3489|   3.5|1419794797|  3.1093388|
|    21| 104841|   4.0|1403460671|   4.708873|
|    21|   8961|   3.5|1441393027|  3.0724843|
|    21|  599